## Human pluripotent stem cells identify molecular targets of trisomy 12 in chronic lymphocytic leukemia patients
#### Jennifer C. Reid et al. Cell Reports, 2021

In [ ]:
# Load required libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
%matplotlib inline

## ANALYSIS FOR ‘ALL GENES’

In [ ]:
dfall = pd.read_csv('KNN_allgenes_tri12_or_not.csv')
dfall.head()

Cytogenetics	A1BG	A1CF	A2M	A2ML1	A3GALT2	...	RNU1-143P	RNU1-146P	RNU1-148P	RNU1-14P	RNU1-150P<br>
0	not	6.87750	5.60989	5.84489	5.59911	5.99098	...	6.22049	3.49924	6.87333	4.67932	6.43522 <br>
1	not	7.17746	5.31488	5.62043	4.95479	6.35395	...	5.23416	3.45891	6.84149	4.69038	6.58324 <br>
2	not	6.91770	5.34324	5.79977	5.29049	5.94090	...	5.36049	3.67363	6.67255	4.50193	6.22993 <br>
3	not	7.16713	5.55926	5.91698	5.76116	6.07529	...	6.19797	3.26875	7.14785	4.59984	6.70062 <br>
not	7.17281	5.56064	5.94240	5.63104	6.01762	...	5.37074	3.66070	6.84350	4.72973	6.60047 <br>
5 rows × 16379 columns

In [ ]:
# QC
print("all genes:",dfall['Cytogenetics'].value_counts())

all genes: not      46 <br>
&emsp; &emsp; &emsp;            tri12    17

In [ ]:
# Define feature set
dfall.columns

Index(['Cytogenetics', 'A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', <br>
&emsp;       ...<br>
&emsp;       'RNU1-139P', 'RNU1-13P', 'RNU1-140P', 'RNU1-141P', 'RNU1-142P',<br>
&emsp;       'RNU1-143P', 'RNU1-146P', 'RNU1-148P', 'RNU1-14P', 'RNU1-150P'],<br>
&emsp;      dtype='object', length=16379)


In [ ]:
Xall = dfall.iloc[:,2:].values
Xall[0:5]

array([[6.8775 , 5.60989, 5.84489, ..., 6.87333, 4.67932, 6.43522],<br>
&emsp;   [7.17746, 5.31488, 5.62043, ..., 6.84149, 4.69038, 6.58324],<br>
&emsp;   [6.9177 , 5.34324, 5.79977, ..., 6.67255, 4.50193, 6.22993],<br>
&emsp;   [7.16713, 5.55926, 5.91698, ..., 7.14785, 4.59984, 6.70062],<br>
&emsp;   [7.17281, 5.56064, 5.9424 , ..., 6.8435 , 4.72973, 6.60047]]<br>

In [ ]:
yall = dfall['Cytogenetics'].values
yall[0:5]

array(['not', 'not', 'not', 'not', 'not'], dtype=object)

In [ ]:
# PCA
pca = PCA(n_components=2)
pca.fit(Xall)
Xall_pca = pca.transform(Xall)
print("original shape:   ", Xall.shape)
print("transformed shape:", Xall_pca.shape)
print("components:", pca.components_)
print("explained variance:", pca.explained_variance_ratio_)

original shape:    (64, 16377)<br>
transformed shape: (64, 2)<br>
components: [[ 0.00558722  0.00398884  0.00382769 ...  0.00271563 -0.00076507 0.00390245]<br>
&emsp; [-0.00674197 -0.00351118 -0.00531889 ... -0.00957618 -0.00392257 -0.00886299]]<br>
explained variance: [0.14239606 0.09949726]<br>

In [ ]:
# export PCA coordinates and plot in Prism 7
Xall_pca_df = pd.DataFrame(Xall_pca)
np.savetxt('Xall_pca_df.txt', Xall_pca_df, fmt='%1.9f')

In [ ]:
# K nearest neighbour analysis
# 70% of dataset used for training and 30% of dataset used for testing (0.3)
# Same random seed as the other analysis
Xall_train, Xall_test, yall_train, yall_test = train_test_split(Xall_pca, yall, test_size=0.3, random_state=5)
print ('Train set:', Xall_train.shape,  yall_train.shape)
print ('Test set:', Xall_test.shape,  yall_test.shape)

Train set: (44, 2) (44,) <br>
Test set: (20, 2) (20,)

In [ ]:
Ks = 10
mean_acc_all = np.zeros((Ks-1))
std_acc_all = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    #Train Model and Predict
    neigh_all = KNeighborsClassifier(n_neighbors = n).fit(Xall_train,yall_train)
    yhatall=neigh_all.predict(Xall_test)
    mean_acc_all[n-1] = metrics.accuracy_score(yall_test, yhatall)
mean_acc_all

array([0.6 , 0.7 , 0.65, 0.65, 0.7 , 0.7 , 0.7 , 0.7 , 0.7 ]) <br>
<br>
Plot accuracy values (mean_acc_all) from k=1 to 9 in Prism

## ANALYSIS FOR ‘TOP CHR. 12 GENES’

In [ ]:
df9 = pd.read_csv('KNN_9genes_tri12_or_not.csv')
df9.head()

	Cytogenetics ALX1 IRAK4 KITLG NTS PTPRB PUS7L SLCO1A2 TBX5 TCP11L2
0	not	5.06446	9.18823	3.41562	3.21892	4.63095	7.86214	4.03589	6.24069	10.90930<br>
1	not	4.92332	8.98104	3.28448	3.53200	4.28280	7.76889	3.49250	5.98155	9.73198<br>
2	not	4.63945	9.46348	3.20656	3.33914	4.47696	8.17006	3.46301	5.85343	10.43220<br>
3	not	4.78123	8.65459	3.23249	3.24703	4.59339	7.90041	3.81321	6.20550	10.69520<br>
4	not	5.11331	8.62725	3.25025	3.32743	4.47847	7.79577	3.47778	6.20558	11.13830<br>

In [ ]:
# QC
print("9 genes:",df9['Cytogenetics'].value_counts())

9 genes: not      46 <br>
&emsp; &emsp; &emsp;         tri12    17

In [ ]:
# Define feature set
df9.columns

Index(['ID', 'Cytogenetics', 'ALX1', 'IRAK4', 'KITLG', 'NTS', 'PTPRB', 'PUS7L', 'SLCO1A2', 'TBX5', 'TCP11L2'], dtype='object')


In [ ]:
X9 = df9[['ALX1', 'IRAK4', 'KITLG', 'NTS', 'PTPRB', 'PUS7L', 'SLCO1A2', 'TBX5', 'TCP11L2']].values  #.astype(float)
X9[0:1]

array([[ 5.06446,  9.18823,  3.41562,  3.21892,  4.63095,  7.86214, 4.03589,  6.24069, 10.9093 ]])

In [ ]:
y9 = df9['Cytogenetics'].values
y9[0:5]

array(['not', 'not', 'not', 'not', 'not'], dtype=object)


In [ ]:
# PCA
pca = PCA(n_components=2)
pca.fit(X9)
X9_pca = pca.transform(X9)
print("original shape:", X9.shape)
print("transformed shape:", X9_pca.shape)
print("components:", pca.components_)
print("explained variance:", pca.explained_variance_ratio_)

original shape: (63, 9)<br>
transformed shape: (63, 2)<br>
components: [[ 0.027831   -0.60026611  0.02656125 -0.02816084  0.10960376 -0.59295226 0.06240777  <br>&emsp; 0.06294761 -0.51569856]<br>
&emsp; [ 0.07142141 -0.34334739  0.12969711  0.09276781  0.18036988 -0.29371034 0.17642951  0.16057406  0.8221155 ]]<br>
explained variance: [0.38132274 0.30949963]

In [ ]:
# export PCA coordinates and plot in Prism 7
X9_pca_df = pd.DataFrame(X9_pca)
np.savetxt('X9_pca_df.txt', X9_pca_df,fmt='%1.9f')
X9_pca_df

In [ ]:
# K nearest neighbour analysis
# 70% of dataset used for training and 30% of dataset used for testing (0.3)
# Same random seed as the other analysis
X9_train, X9_test, y9_train, y9_test = train_test_split(X9_pca, y9, test_size=0.3, random_state=5)
print ('Train set:', X9_train.shape,  y9_train.shape)
print ('Test set:', X9_test.shape,  y9_test.shape)

Train set: (44, 2) (44,) <br>
Test set: (19, 2) (19,)

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    #Train Model and Predict
    neigh9 = KNeighborsClassifier(n_neighbors = n).fit(X9_train,y9_train)
    yhat9=neigh9.predict(X9_test)
    mean_acc[n-1] = metrics.accuracy_score(y9_test, yhat9)
mean_acc

array([0.73684211, 0.78947368, 0.84210526, 0.78947368, 0.89473684, <br>
&emsp;       0.84210526, 0.84210526, 0.78947368, 0.78947368])
<br>
Plot accuracy values (mean_acc) from k=1 to 9 in Prism